## 연관 광고 상품 모형 구축
### 1. 데이터 불러오기 및 전처리

In [ ]:
# 라이브러리 설치
import copy
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
import pandas as pd
import pymysql

# 그래프 관련 패키지
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font', family = 'NANUMBARUNGOTHIC')
print(plt.rcParams['font.family'])

%matplotlib inline

['NANUMBARUNGOTHIC']


In [ ]:
# 자료를 어느 날짜까지 가져올지?
endDate = '2020-11-01'

In [ ]:
# DB 접속(정보 생략)
conn = pymysql.connect()

In [ ]:
main_query = "SELECT reportName, mediaId, mediaSlotId, MIN(statDate) AS startDate, MAX(statDate) AS endDate, SUM(amtExecution) AS adCost, SUM(impressionCnt) AS impressionCnt, SUM(clickCnt) AS clickCnt, SUM(viewCnt) AS viewCnt,\
                    cmBrandName, cmCompanyName, cmBusinessTypeName, cmCategoryName, cmBrandId, cmCompanyId, cmBusinessTypeId, cmCategoryId FROM mainAdStatDaily\
            GROUP BY reportName, mediaId, mediaSlotId"

other_query = "SELECT reportName, mediaId, mediaSlotId, MIN(statDate) AS startDate, MAX(statDate) AS endDate, adCost, SUM(impressionCnt) AS impressionCnt, SUM(clickCnt) AS clickCnt, SUM(viewCnt) AS viewCnt,\
                    cmBrandName, cmCompanyName, cmBusinessTypeName, cmCategoryName, cmBrandId, cmCompanyId, cmBusinessTypeId, cmCategoryId\
            FROM (\
            SELECT t1.reportName, t1.mediaId, t1.mediaSlotId, t1.statDate, adCost, SUM(impressionCnt) AS impressionCnt, SUM(clickCnt) AS clickCnt, SUM(viewCnt) AS viewCnt, t3.* FROM datacenterdb.otherAdStatDaily t1\
            LEFT OUTER JOIN rainbow.Campaign AS t2 ON t1.cmCampaignId = t2.campaignId \
            LEFT OUTER JOIN (SELECT DISTINCT brandName AS cmBrandName, companyName AS cmCompanyName, businessTypeName AS cmBusinessTypeName, categoryName AS cmCategoryName, \
                            brandId AS cmBrandId, t1.companyId AS cmCompanyId, t2.businessTypeId AS cmBusinessTypeId, t1.categoryId AS cmCategoryId\
                    FROM rainbow.Brand t1, rainbow.BusinessCategory t2, rainbow.BusinessType t3, rainbow.Company t4\
                    WHERE t1.categoryId=t2.categoryId AND t2.businessTypeId=t3.businessTypeId AND t1.companyId=t4.companyId) AS t3 ON t2.brandId = t3.cmBrandId\
            GROUP BY t1.reportName, t1.mediaId, t1.statDate, t1.mediaSlotId) t4\
            GROUP BY reportName, mediaId, mediaSlotId"

media_query = "SELECT t1.mediaName, t3.slotName, t1.mediaId, t3.mediaSlotId\
            FROM (SELECT * FROM rainbow.Media WHERE deleteFlag = 'N') AS t1\
            LEFT OUTER JOIN (SELECT * FROM rainbow.MediaSection WHERE deleteFlag = 'N') AS t2 ON t1.mediaId = t2.mediaId\
            LEFT OUTER JOIN (SELECT * FROM rainbow.MediaSlot WHERE deleteFlag = 'N') AS t3 ON t2.mediaSectionId = t3.mediaSectionId\
            ORDER BY t1.mediaId, t3.mediaSlotId"

main_df = pd.read_sql(main_query, conn)
other_df = pd.read_sql(other_query, conn)
media_df = pd.read_sql(media_query, conn)

print(main_df.shape) # 1,307*17
print(other_df.shape) # 666*17
print(media_df.shape) # 2,392*4

(1307, 17)
(666, 17)
(2392, 4)


In [ ]:
# 날짜 포맷 변경
main_df['startDate'] = pd.to_datetime(main_df['startDate']).dt.strftime("%Y-%m-%d").astype('datetime64[ns]')
main_df['endDate'] = pd.to_datetime(main_df['endDate']).dt.strftime("%Y-%m-%d").astype('datetime64[ns]')

other_df['startDate'] = pd.to_datetime(other_df['startDate']).astype('datetime64[ns]')
other_df['endDate'] = pd.to_datetime(other_df['endDate']).astype('datetime64[ns]')

main_df = main_df[main_df['endDate'] < endDate]
other_df = other_df[other_df['endDate'] < endDate]

print(main_df.shape) # 1,306*17
print(other_df.shape) # 666*17

(1306, 17)
(666, 17)


In [ ]:
# 주요 매체 + 기타 매체
mix_df = pd.concat([main_df, other_df])
print(mix_df.shape) # 1,972*17
mix_df.head()

(1972, 17)


,reportName,mediaId,mediaSlotId,startDate,endDate,adCost,impressionCnt,clickCnt,viewCnt,cmBrandName,cmCompanyName,cmBusinessTypeName,cmCategoryName,cmBrandId,cmCompanyId,cmBusinessTypeId,cmCategoryId
0,(Crossmedia) AIG손해보험 캠페인_Report_1908019,28,4971,2019-08-12,2019-08-18,2400000.0,797520.0,2325.0,0.0,AIG손해보험,AIG손해보험,"금융, 보험 및 증권",금융 및 보험,2336,2781,6,34
1,(Crossmedia) 이플러스코리아_오스트레일리안골드 캠페인_Report_2007...,897,6598,2020-07-13,2020-07-19,2196074.0,2081133.0,3035.0,0.0,이플러스코리아,이플러스코리아,화장품 및 보건용품,화장품 및 보건용품 기타,3559,3998,19,178
2,(Crossmedia) 플랜코리아 10월 대희가 필요해 캠페인_Final Repor...,888,5612,2020-10-16,2020-10-28,349163.0,20388.0,133.0,4174.0,플랜코리아,플랜코리아,관공서 및 단체,단체,1012,1345,21,185
3,(Crossmedia) 플랜코리아 8월 깁미파이브 2차 캠페인_Final Repor...,888,5612,2020-08-20,2020-08-26,346726.0,59010.0,163.0,21761.0,플랜코리아,플랜코리아,관공서 및 단체,단체,1012,1345,21,185
4,(Crossmedia) 플랜코리아 8월 깁미파이브 3차 캠페인_Final Repor...,888,5612,2020-08-28,2020-09-03,342378.0,25293.0,72.0,14155.0,플랜코리아,플랜코리아,관공서 및 단체,단체,1012,1345,21,185


In [ ]:
# 매체 정보 추가
media_df['mediaSlotName'] = media_df['mediaName'] + " / " + media_df['slotName']

mix_df['mediaId'] = np.where((mix_df.mediaId == '897') & (mix_df.mediaSlotId == '5824'), '896', mix_df.mediaId) # 주요 매체 매핑 로직에 의해 별도 처리
mix_df = mix_df.astype({'mediaId': int, 'mediaSlotId': int})

mix_df = pd.merge(mix_df, media_df, how = 'left', on = ['mediaId', 'mediaSlotId'])
mix_df.head()

,reportName,mediaId,mediaSlotId,startDate,endDate,adCost,impressionCnt,clickCnt,viewCnt,cmBrandName,cmCompanyName,cmBusinessTypeName,cmCategoryName,cmBrandId,cmCompanyId,cmBusinessTypeId,cmCategoryId,mediaName,slotName,mediaSlotName
0,(Crossmedia) AIG손해보험 캠페인_Report_1908019,28,4971,2019-08-12,2019-08-18,2400000.0,797520.0,2325.0,0.0,AIG손해보험,AIG손해보험,"금융, 보험 및 증권",금융 및 보험,2336,2781,6,34,네이버(NOSP),M_메인_여행_브랜딩DA,네이버(NOSP) / M_메인_여행_브랜딩DA
1,(Crossmedia) 이플러스코리아_오스트레일리안골드 캠페인_Report_2007...,897,6598,2020-07-13,2020-07-19,2196074.0,2081133.0,3035.0,0.0,이플러스코리아,이플러스코리아,화장품 및 보건용품,화장품 및 보건용품 기타,3559,3998,19,178,네이버(GFA),스마트채널,네이버(GFA) / 스마트채널
2,(Crossmedia) 플랜코리아 10월 대희가 필요해 캠페인_Final Repor...,888,5612,2020-10-16,2020-10-28,349163.0,20388.0,133.0,4174.0,플랜코리아,플랜코리아,관공서 및 단체,단체,1012,1345,21,185,구글(유튜브)_와이즈버즈,트루뷰 인스트림,구글(유튜브)_와이즈버즈 / 트루뷰 인스트림
3,(Crossmedia) 플랜코리아 8월 깁미파이브 2차 캠페인_Final Repor...,888,5612,2020-08-20,2020-08-26,346726.0,59010.0,163.0,21761.0,플랜코리아,플랜코리아,관공서 및 단체,단체,1012,1345,21,185,구글(유튜브)_와이즈버즈,트루뷰 인스트림,구글(유튜브)_와이즈버즈 / 트루뷰 인스트림
4,(Crossmedia) 플랜코리아 8월 깁미파이브 3차 캠페인_Final Repor...,888,5612,2020-08-28,2020-09-03,342378.0,25293.0,72.0,14155.0,플랜코리아,플랜코리아,관공서 및 단체,단체,1012,1345,21,185,구글(유튜브)_와이즈버즈,트루뷰 인스트림,구글(유튜브)_와이즈버즈 / 트루뷰 인스트림


In [ ]:
# Null값 체크
mix_df.isnull().sum()

reportName            0
mediaId               0
mediaSlotId           0
startDate             0
endDate               0
adCost                0
impressionCnt         0
clickCnt              0
viewCnt               0
cmBrandName           0
cmCompanyName         0
cmBusinessTypeName    0
cmCategoryName        0
cmBrandId             0
cmCompanyId           0
cmBusinessTypeId      0
cmCategoryId          0
mediaName             5
slotName              5
mediaSlotName         5
dtype: int64

In [ ]:
# 삭제된 매체 or 슬롯 ID로 등록된 경우(체크하여 ID 변경 필요!)
media_null = mix_df[mix_df.mediaName.isnull()]
media_null = media_null[['mediaId', 'mediaSlotId']].drop_duplicates()
media_null = media_null.sort_values(by = ['mediaId', 'mediaSlotId'], ascending = True).reset_index(drop = True)

print(len(media_null)) # 5
media_null

5


,mediaId,mediaSlotId
0,3,5832
1,3,5833
2,3,5834
3,70,1323
4,645,6101


In [ ]:
# 삭제된 매체 ID를 가진 로우는 제외
mix_df = mix_df[mix_df.mediaId != 3]
mix_df = mix_df[mix_df.mediaId != 70]
mix_df = mix_df[mix_df.mediaId != 645]

print(mix_df.shape) # 1,967*20

(1967, 20)


In [ ]:
# 노출수가 0인 상품 제외(offerwall, CPL 등)
# print(len(mix_df[mix_df.impressionCnt == 0])) # 38

# mix_df = mix_df[mix_df.impressionCnt != 0]
# print(mix_df.shape) # 1,842*20

In [ ]:
# 캠페인 효율, 단가 변수 생성
# mix_df = mix_df[mix_df.impressionCnt > mix_df.clickCnt]
# mix_df = mix_df[mix_df.impressionCnt > mix_df.viewCnt]

mix_df['ctr'] = np.where(mix_df['impressionCnt'] == 0, 0, mix_df['clickCnt'] / mix_df['impressionCnt'] * 100)
mix_df['vtr'] = np.where(mix_df['impressionCnt'] == 0, 0, mix_df['viewCnt'] / mix_df['impressionCnt'] * 100)

mix_df['cpm'] = np.where(mix_df['impressionCnt'] == 0, 0, mix_df['adCost'] / mix_df['impressionCnt'] * 1000)
mix_df['cpc'] = np.where(mix_df['clickCnt'] == 0, 0, mix_df['adCost'] / mix_df['clickCnt'])
mix_df['cpv'] = np.where(mix_df['viewCnt'] == 0, 0, mix_df['adCost'] / mix_df['viewCnt'])

In [ ]:
print(mix_df.shape) # 1,967*25
mix_df.head() # key 변수: 캠페인, 매체, 상품

(1967, 25)


,reportName,mediaId,mediaSlotId,startDate,endDate,adCost,impressionCnt,clickCnt,viewCnt,cmBrandName,...,cmBusinessTypeId,cmCategoryId,mediaName,slotName,mediaSlotName,ctr,vtr,cpm,cpc,cpv
0,(Crossmedia) AIG손해보험 캠페인_Report_1908019,28,4971,2019-08-12,2019-08-18,2400000.0,797520.0,2325.0,0.0,AIG손해보험,...,6,34,네이버(NOSP),M_메인_여행_브랜딩DA,네이버(NOSP) / M_메인_여행_브랜딩DA,0.291529,0.000000,3009.328920,1032.258065,0.000000
1,(Crossmedia) 이플러스코리아_오스트레일리안골드 캠페인_Report_2007...,897,6598,2020-07-13,2020-07-19,2196074.0,2081133.0,3035.0,0.0,이플러스코리아,...,19,178,네이버(GFA),스마트채널,네이버(GFA) / 스마트채널,0.145834,0.000000,1055.230012,723.582867,0.000000
2,(Crossmedia) 플랜코리아 10월 대희가 필요해 캠페인_Final Repor...,888,5612,2020-10-16,2020-10-28,349163.0,20388.0,133.0,4174.0,플랜코리아,...,21,185,구글(유튜브)_와이즈버즈,트루뷰 인스트림,구글(유튜브)_와이즈버즈 / 트루뷰 인스트림,0.652345,20.472827,17125.907397,2625.285714,83.651893
3,(Crossmedia) 플랜코리아 8월 깁미파이브 2차 캠페인_Final Repor...,888,5612,2020-08-20,2020-08-26,346726.0,59010.0,163.0,21761.0,플랜코리아,...,21,185,구글(유튜브)_와이즈버즈,트루뷰 인스트림,구글(유튜브)_와이즈버즈 / 트루뷰 인스트림,0.276224,36.876801,5875.715980,2127.153374,15.933367
4,(Crossmedia) 플랜코리아 8월 깁미파이브 3차 캠페인_Final Repor...,888,5612,2020-08-28,2020-09-03,342378.0,25293.0,72.0,14155.0,플랜코리아,...,21,185,구글(유튜브)_와이즈버즈,트루뷰 인스트림,구글(유튜브)_와이즈버즈 / 트루뷰 인스트림,0.284664,55.964101,13536.472542,4755.250000,24.187778


## -------------------------------------------------------------------------------------------------------------------------------
### 2. 동시 집행 상품(연관성 분석)

In [ ]:
# Transaction 데이터 생성(캠페인별 집행 상품)
tr_df = pd.get_dummies(mix_df['mediaSlotId'])
tr_df2 = pd.concat([mix_df['reportName'], tr_df], axis = 1).reset_index(drop = True)

tr_df_sum = tr_df2.groupby(['reportName']).sum()
tr_df_sum2 = np.where(tr_df_sum >= 1, 1, 0)

tr_df_col = tr_df_sum.columns.values # 슬롯 ID
tr_df_row = tr_df_sum.index.values # 캠페인명

tr_df_sum2 = pd.DataFrame(tr_df_sum2, index = tr_df_row, columns = tr_df_col)
tr_df_sum2.columns = tr_df_sum2.columns.astype(int).astype(str)

print(tr_df_sum2.shape) # 캠페인 수: 909 / 상품 수: 285
tr_df_sum2.head()

(909, 285)


,290,988,2275,2277,2474,2475,2476,2890,3126,3143,...,6832,6835,6836,6875,6961,6962,6963,6964,6966,6967
(Crossmedia) AIG손해보험 캠페인_Report_1908019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(Crossmedia) 롯데관광 8월 캠페인_Report_190902,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(Crossmedia) 이플러스코리아_예스포러브 캠페인_Report_200901_F_18263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(Crossmedia) 이플러스코리아_오스트레일리안골드 캠페인_Report_200720_F_17718,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
(Crossmedia) 풀무원 잇슬림 2월 캠페인_Final Report_0302_16349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 연관 상품 테이블 생성
list_of_df = []
list_of_df2 = []

for pdFilter in tr_df_sum2.columns:

    tr_df_temp = tr_df_sum2.loc[(tr_df_sum2[pdFilter] == 1), :] # 특정 상품 데이터셋 생성
    
    # 상품별 동시 집행 횟수(전체)
    tr_df_temp_cnt = tr_df_temp.sum(axis = 0)
    tr_df_temp_cnt = tr_df_temp_cnt.values.reshape(len(tr_df_temp_cnt), 1) # dimension이 있어야 연산 가능
    tr_df_temp_cnt = pd.DataFrame(tr_df_temp_cnt, index = tr_df_col, columns = ['cnt']) # index, column 명칭 지정
    
    tr_df_temp_cnt_new = tr_df_temp_cnt.copy()
    tr_df_temp_cnt_new.reset_index(inplace = True)
    
    tr_df_temp_cnt_new['slot_idx'] = pdFilter
    tr_df_temp_cnt_new = tr_df_temp_cnt_new.rename(columns = {'index': 'related_slot_idx'})
    tr_df_temp_cnt_new = tr_df_temp_cnt_new[['slot_idx', 'related_slot_idx', 'cnt']]
    
    list_of_df.append(tr_df_temp_cnt_new)
    
    # 상품별 동시 집행 횟수 TOP5
    tr_df_temp_cnt2 = tr_df_temp_cnt.sort_values('cnt', ascending = False)[1:6]
    tr_df_temp_cnt2 = tr_df_temp_cnt2[tr_df_temp_cnt2.cnt != 0] # 동시 집행한 적 없는 상품 제외
    tr_df_temp_cnt2.reset_index(inplace = True)
    
    tr_df_temp_cnt2['slot_idx'] = pdFilter
    tr_df_temp_cnt2 = tr_df_temp_cnt2.rename(columns = {'index': 'related_slot_idx'})
    tr_df_temp_cnt2 = tr_df_temp_cnt2[['slot_idx', 'related_slot_idx', 'cnt']]
    
    list_of_df2.append(tr_df_temp_cnt2)

rl_df_total = pd.concat(list_of_df)    
rl_df = pd.concat(list_of_df2)

In [ ]:
print(rl_df_total.shape) # 81,225*3
print(rl_df.shape) # 1,064*3
rl_df

(81225, 3)
(1064, 3)


,slot_idx,related_slot_idx,cnt
0,988,3404,1
1,988,6256,1
2,988,6018,1
3,988,6016,1
4,988,6257,1
...,...,...,...
2,6962,6836,1
3,6962,6835,1
4,6962,6832,1
0,6963,6963,1


In [ ]:
print("연관 상품 노출 비율:", len(rl_df.slot_idx.unique())/len(tr_df_sum2.columns)) # 91.6%

연관 상품 노출 비율: 0.9157894736842105


In [ ]:
# rl_df.to_csv('/home/anaconda3/da/data/related_slot_many_2010.csv') # 파일명 변경(데이터 최종 월)

## -------------------------------------------------------------------------------------------------------------------------------
### 3. 효율 상승 상품

#### 1) 노출수 최대화

In [ ]:
list_of_cpm = []

for pdFilter in tr_df_sum2.columns:
    
    # 평균 CPM보다 낮은 캠페인
    pdFilter = pd.to_numeric(pdFilter)
    mix_df_temp = mix_df[mix_df.mediaSlotId == pdFilter] # 특정 상품 데이터셋 생성
    mix_df_temp = mix_df_temp[mix_df_temp.impressionCnt != 0]
    
    if len(mix_df_temp) == 0: 
        continue
    
    mix_df_cpm_mean = mix_df_temp['adCost'].sum() / mix_df_temp['impressionCnt'].sum() * 1000 # 평균 CPM
    mix_df_cpm = mix_df_temp[mix_df_temp.cpm < mix_df_cpm_mean]
    mix_df_cpm_report = mix_df_cpm['reportName'].tolist() # CPM 낮은 캠페인 
    
    # CPM 낮은 캠페인별 집행 상품
    tr_df_cpm = tr_df_sum2.loc[mix_df_cpm_report,]

    slot_cnt_cpm = tr_df_cpm.sum(axis = 0)
    slot_cnt_cpm = slot_cnt_cpm.values.reshape(len(slot_cnt_cpm), 1) # dimension이 있어야 연산 가능
    slot_cnt_cpm = pd.DataFrame(slot_cnt_cpm, index = tr_df_col, columns = ['cnt']) # index, column 명칭 지정   
    slot_cnt_cpm = slot_cnt_cpm[slot_cnt_cpm.cnt != 0] # 동시 집행한 적 없는 상품 제외

    slot_cnt_cpm.reset_index(inplace = True)
    slot_cnt_cpm['slot_idx'] = pdFilter
    slot_cnt_cpm = slot_cnt_cpm.rename(columns = {'index': 'related_slot_idx'})

    slot_cnt_cpm = slot_cnt_cpm[slot_cnt_cpm.related_slot_idx != pdFilter]  # 해당 상품 제외
    
    # 노출수를 높여주는 상품 TOP5
    pdFilter = str(pdFilter)

    rl_df_total_temp = rl_df_total[rl_df_total.slot_idx == pdFilter]
    rl_df_total_temp = rl_df_total_temp.rename(columns = {'cnt': 'total_cnt'})
    rl_df_total_temp.slot_idx = pd.to_numeric(rl_df_total_temp.slot_idx)

    slot_cnt_cpm2 = pd.merge(slot_cnt_cpm, rl_df_total_temp, how = 'left', on = ['slot_idx', 'related_slot_idx'])
    slot_cnt_cpm2['cnt_ratio'] = slot_cnt_cpm2['cnt'] / slot_cnt_cpm2['total_cnt'] # 상위권 랭크인 비율
    slot_cnt_cpm2 = slot_cnt_cpm2[['slot_idx', 'related_slot_idx', 'total_cnt', 'cnt', 'cnt_ratio']]

    slot_cnt_cpm2_temp = slot_cnt_cpm2[slot_cnt_cpm2.total_cnt >= 2] # 우연히 랭크인된 상품 제외(1번 랭크인)
    slot_cnt_cpm2_temp = slot_cnt_cpm2_temp.sort_values('cnt_ratio', ascending = False)[:5]
    
    list_of_cpm.append(slot_cnt_cpm2_temp)
 
rl_df_cpm = pd.concat(list_of_cpm)

In [ ]:
print(rl_df_cpm.shape) # 362*5
rl_df_cpm

(362, 5)


,slot_idx,related_slot_idx,total_cnt,cnt,cnt_ratio
0,2275,2277,2,1,0.500000
1,2275,2890,2,1,0.500000
3,2275,4998,2,1,0.500000
4,2275,5029,2,1,0.500000
5,2275,5620,2,1,0.500000
...,...,...,...,...,...
6,6767,6673,4,2,0.500000
0,6768,5612,6,2,0.333333
1,6768,6767,5,1,0.200000
0,6770,5612,2,1,0.500000


In [ ]:
print("노출수 최대화 상품 노출 비율:", len(rl_df_cpm.slot_idx.unique())/len(tr_df_sum2.columns)) # 37.2%

노출수 최대화 상품 노출 비율: 0.3719298245614035


#### 2) 클릭수 최대화

In [ ]:
list_of_cpc = []

for pdFilter in tr_df_sum2.columns:
    
    # 평균 CPC보다 낮은 캠페인
    pdFilter = pd.to_numeric(pdFilter)
    mix_df_temp = mix_df[mix_df.mediaSlotId == pdFilter] # 특정 상품 데이터셋 생성
    mix_df_temp = mix_df_temp[mix_df_temp.clickCnt != 0]
    
    if len(mix_df_temp) == 0: 
        continue
    
    mix_df_cpc_mean = mix_df_temp['adCost'].sum() / mix_df_temp['clickCnt'].sum() # 평균 CPC
    mix_df_cpc = mix_df_temp[mix_df_temp.cpc < mix_df_cpc_mean]
    mix_df_cpc_report = mix_df_cpc['reportName'].tolist() # CPC 낮은 캠페인 
    
    # CPC 낮은 캠페인별 집행 상품
    tr_df_cpc = tr_df_sum2.loc[mix_df_cpc_report,]

    slot_cnt_cpc = tr_df_cpc.sum(axis = 0)
    slot_cnt_cpc = slot_cnt_cpc.values.reshape(len(slot_cnt_cpc), 1) # dimension이 있어야 연산 가능
    slot_cnt_cpc = pd.DataFrame(slot_cnt_cpc, index = tr_df_col, columns = ['cnt']) # index, column 명칭 지정   
    slot_cnt_cpc = slot_cnt_cpc[slot_cnt_cpc.cnt != 0] # 동시 집행한 적 없는 상품 제외

    slot_cnt_cpc.reset_index(inplace = True)
    slot_cnt_cpc['slot_idx'] = pdFilter
    slot_cnt_cpc = slot_cnt_cpc.rename(columns = {'index': 'related_slot_idx'})

    slot_cnt_cpc = slot_cnt_cpc[slot_cnt_cpc.related_slot_idx != pdFilter]  # 해당 상품 제외
    
    # 클릭수를 높여주는 상품 TOP5
    pdFilter = str(pdFilter)

    rl_df_total_temp = rl_df_total[rl_df_total.slot_idx == pdFilter]
    rl_df_total_temp = rl_df_total_temp.rename(columns = {'cnt': 'total_cnt'})
    rl_df_total_temp.slot_idx = pd.to_numeric(rl_df_total_temp.slot_idx)

    slot_cnt_cpc2 = pd.merge(slot_cnt_cpc, rl_df_total_temp, how = 'left', on = ['slot_idx', 'related_slot_idx'])
    slot_cnt_cpc2['cnt_ratio'] = slot_cnt_cpc2['cnt'] / slot_cnt_cpc2['total_cnt'] # 상위권 랭크인 비율
    slot_cnt_cpc2 = slot_cnt_cpc2[['slot_idx', 'related_slot_idx', 'total_cnt', 'cnt', 'cnt_ratio']]

    slot_cnt_cpc2_temp = slot_cnt_cpc2[slot_cnt_cpc2.total_cnt >= 2] # 우연히 랭크인된 상품 제외(1번 랭크인)
    slot_cnt_cpc2_temp = slot_cnt_cpc2_temp.sort_values('cnt_ratio', ascending = False)[:5]
    
    list_of_cpc.append(slot_cnt_cpc2_temp)
 
rl_df_cpc = pd.concat(list_of_cpc)

In [ ]:
print(rl_df_cpc.shape) # 365*5
rl_df_cpc

(365, 5)


,slot_idx,related_slot_idx,total_cnt,cnt,cnt_ratio
0,2275,2277,2,1,0.500000
1,2275,2890,2,1,0.500000
3,2275,4998,2,1,0.500000
4,2275,5029,2,1,0.500000
5,2275,5620,2,1,0.500000
...,...,...,...,...,...
11,6767,5612,7,6,0.857143
1,6768,6673,2,1,0.500000
2,6768,6767,5,2,0.400000
0,6768,5612,6,2,0.333333


In [ ]:
print("클릭수 최대화 상품 노출 비율:", len(rl_df_cpc.slot_idx.unique())/len(tr_df_sum2.columns)) # 38.2%

클릭수 최대화 상품 노출 비율: 0.3824561403508772


#### 3) 조회수 최대화

In [ ]:
list_of_cpv = []

for pdFilter in tr_df_sum2.columns:
    
    # 평균 CPV보다 낮은 캠페인
    pdFilter = pd.to_numeric(pdFilter)
    mix_df_temp = mix_df[mix_df.mediaSlotId == pdFilter] # 특정 상품 데이터셋 생성
    mix_df_temp = mix_df_temp[mix_df_temp.viewCnt != 0]
    
    if len(mix_df_temp) == 0: 
        continue
    
    mix_df_cpv_mean = mix_df_temp['adCost'].sum() / mix_df_temp['viewCnt'].sum() # 평균 CPV
    mix_df_cpv = mix_df_temp[mix_df_temp.cpv < mix_df_cpv_mean]
    mix_df_cpv_report = mix_df_cpv['reportName'].tolist() # CPV 낮은 캠페인 
    
    # CPV 낮은 캠페인별 집행 상품
    tr_df_cpv = tr_df_sum2.loc[mix_df_cpv_report,]

    slot_cnt_cpv = tr_df_cpv.sum(axis = 0)
    slot_cnt_cpv = slot_cnt_cpv.values.reshape(len(slot_cnt_cpv), 1) # dimension이 있어야 연산 가능
    slot_cnt_cpv = pd.DataFrame(slot_cnt_cpv, index = tr_df_col, columns = ['cnt']) # index, column 명칭 지정   
    slot_cnt_cpv = slot_cnt_cpv[slot_cnt_cpv.cnt != 0] # 동시 집행한 적 없는 상품 제외

    slot_cnt_cpv.reset_index(inplace = True)
    slot_cnt_cpv['slot_idx'] = pdFilter
    slot_cnt_cpv = slot_cnt_cpv.rename(columns = {'index': 'related_slot_idx'})

    slot_cnt_cpv = slot_cnt_cpv[slot_cnt_cpv.related_slot_idx != pdFilter]  # 해당 상품 제외
    
    # 클릭수를 높여주는 상품 TOP5
    pdFilter = str(pdFilter)

    rl_df_total_temp = rl_df_total[rl_df_total.slot_idx == pdFilter]
    rl_df_total_temp = rl_df_total_temp.rename(columns = {'cnt': 'total_cnt'})
    rl_df_total_temp.slot_idx = pd.to_numeric(rl_df_total_temp.slot_idx)

    slot_cnt_cpv2 = pd.merge(slot_cnt_cpv, rl_df_total_temp, how = 'left', on = ['slot_idx', 'related_slot_idx'])
    slot_cnt_cpv2['cnt_ratio'] = slot_cnt_cpv2['cnt'] / slot_cnt_cpv2['total_cnt'] # 상위권 랭크인 비율
    slot_cnt_cpv2 = slot_cnt_cpv2[['slot_idx', 'related_slot_idx', 'total_cnt', 'cnt', 'cnt_ratio']]

    slot_cnt_cpv2_temp = slot_cnt_cpv2[slot_cnt_cpv2.total_cnt >= 2] # 우연히 랭크인된 상품 제외(1번 랭크인)
    slot_cnt_cpv2_temp = slot_cnt_cpv2_temp.sort_values('cnt_ratio', ascending = False)[:5]
    
    list_of_cpv.append(slot_cnt_cpv2_temp)
 
rl_df_cpv = pd.concat(list_of_cpv)

In [ ]:
print(rl_df_cpv.shape) # 107*5
rl_df_cpv

(107, 5)


,slot_idx,related_slot_idx,total_cnt,cnt,cnt_ratio
0,4955,4961,8,1,0.125000
2,4955,6271,22,1,0.045455
1,5537,4959,3,1,0.333333
8,5540,5612,2,2,1.000000
2,5612,2890,3,3,1.000000
...,...,...,...,...,...
6,6767,6673,4,2,0.500000
0,6768,5612,6,2,0.333333
1,6768,6767,5,1,0.200000
0,6770,5612,2,1,0.500000


In [ ]:
print("조회수 최대화 상품 노출 비율:", len(rl_df_cpv.slot_idx.unique())/len(tr_df_sum2.columns)) # 10.9%

조회수 최대화 상품 노출 비율: 0.10877192982456141


#### 4) Total  효율

In [ ]:
rl_df_cpm['related_key'] = 'imp'
rl_df_cpm2 = rl_df_cpm[['slot_idx', 'related_slot_idx', 'related_key']]

rl_df_cpc['related_key'] = 'click'
rl_df_cpc2 = rl_df_cpc[['slot_idx', 'related_slot_idx', 'related_key']]

rl_df_cpv['related_key'] = 'view'
rl_df_cpv2 = rl_df_cpv[['slot_idx', 'related_slot_idx', 'related_key']]

In [ ]:
rl_df_efficiency = pd.concat([rl_df_cpm2, rl_df_cpc2, rl_df_cpv2])
print(rl_df_efficiency.shape) # 834*3
rl_df_efficiency.head()

(834, 3)


,slot_idx,related_slot_idx,related_key
0,2275,2277,imp
1,2275,2890,imp
3,2275,4998,imp
4,2275,5029,imp
5,2275,5620,imp


In [ ]:
# rl_df_efficiency.to_csv('/home/anaconda3/da/data/related_slot_efficiency_2010.csv') # 파일명 변경(데이터 최종 월)

In [ ]:
# 동시 집행 + 효율 상승 상품 데이터셋 결합
rl_df.slot_idx = pd.to_numeric(rl_df.slot_idx)
rl_df_many = rl_df[['slot_idx', 'related_slot_idx']]

rl_df_total = pd.merge(rl_df_many, rl_df_efficiency, how = 'left', on = ['slot_idx', 'related_slot_idx'])

rl_df_total = rl_df_total.drop_duplicates(['slot_idx', 'related_slot_idx'], keep='first')

In [ ]:
print(rl_df_total.shape) # 1,064*3
rl_df_total.head()

(1064, 3)


,slot_idx,related_slot_idx,related_key
0,988,3404,NaN
1,988,6256,NaN
2,988,6018,NaN
3,988,6016,NaN
4,988,6257,NaN


In [ ]:
print("동시 집행 상품 ONLY:", len(rl_df_total[rl_df_total.related_key.isnull()]) / len(rl_df_total)) # 73%
print("동시 집행 + 효율 상승 상품:", len(rl_df_total[rl_df_total.related_key.notnull()])/ len(rl_df_total)) # 27%

동시 집행 상품 ONLY: 0.7443609022556391
동시 집행 + 효율 상승 상품: 0.2556390977443609


In [ ]:
# related_key_many, related_key_efficiency 변수 생성
rl_df_total['related_key_many'] = 'Y'
rl_df_total['related_key_efficiency'] = np.where(rl_df_total.related_key.notnull(), 'Y', 'N')

rl_df_total = rl_df_total.drop(['related_key'], axis=1)

In [ ]:
rl_df_total

,slot_idx,related_slot_idx,related_key_many,related_key_efficiency
0,988,3404,Y,N
1,988,6256,Y,N
2,988,6018,Y,N
3,988,6016,Y,N
4,988,6257,Y,N
...,...,...,...,...
1271,6962,6836,Y,N
1272,6962,6835,Y,N
1273,6962,6832,Y,N
1274,6963,6963,Y,N


In [ ]:
# DB 접속(정보 생략)
conn2 = pymysql.connect()

In [ ]:
# DB 올리기
# rl_df_total_tp = [tuple(x) for x in rl_df_total.to_numpy()] # 튜플 형태로 변경

# # curs = conn2.cursor()
# # save_query = "TRUNCATE TABLE Slot_Related"
# # curs.execute(save_query)

# save_query2 = """INSERT INTO Slot_Related(slot_idx, related_slot_idx, related_key_many, related_key_efficiency)
#     VALUES (%s, %s, %s, %s)"""
# curs.executemany(save_query2, rl_df_total_tp)

# conn2.commit()
# conn2.close()